In [23]:
from carbon import CarbonSimulatorUI

# Carbon Simulation - Demo 2-1

In this demo we look at **single-curve bidirectional liquidity ("strategies") with a zero width range**

Set up the similation, with the pair USDCETH as default

In [24]:
Sim = CarbonSimulatorUI(pair="ETHUSDC", verbose=False, raiseonerror=False)
Sim

CarbonSimulatorUI(<0 positions, 0 trades>, pair='ETHUSDC')

In [25]:
help(Sim.add_strategy)

Help on method add_strategy in module carbon.simulators.carbon_simulator:

add_strategy(tkn: str, amt_sell: Any, psell_start: Any, psell_end: Any, amt_buy: Any, pbuy_start: Any, pbuy_end: Any, pair: str = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    adds two linked position (one buy, one sell; aka a "strategy")
    
    :tkn:           the token that is sold in the range psell_start/2, eg "ETH"*
    :amt_sell:      the amount of `tkn` that is available for sale in range psell_start/psell_end
    :psell_start:   start of the sell `tkn` range*, quoted in the price convention of `pair`
    :psell_end:     ditto end
    :amt_buy:       the amount of the other token that is available for selling against tkn in range pbuy_start/2
    :pbuy_start:    start of the of the buy `tkn` range*, quoted in the price convention of `pair`
    :pbuy_end:      ditto end
    :pair:          the token pair to which the position corresponds, eg "ETHUSD"


We assert that we can not add an unrelated position to this pair, either using an existing token like LINK, or a token like DNE (does not exist).

In [26]:
Sim.add_strategy("LINK", 10, 2000, 2000, 0, 1000, 1000)

{'success': False,
 'error': "('Invalid token specification (tkn not part of isopair)', 'ETHUSDC', 'LINK')",
 'exception': ValueError('Invalid token specification (tkn not part of isopair)',
            'ETHUSDC',
            'LINK')}

In [27]:
Sim.add_strategy("DNE", 10, 2000, 2000, 0, 1000, 1000)

{'success': False,
 'error': "('Invalid token specification (tkn not part of isopair)', 'ETHUSDC', 'DNE')",
 'exception': ValueError('Invalid token specification (tkn not part of isopair)',
            'ETHUSDC',
            'DNE')}

We set up a single curve where the AMM sells ETH against USDC at 2000, and buys it back at 1000. Note the `linked_to_id` flag that links those two curves; more specifically, it implies that the tokens bought on one curve are available for sale on the other.

In [28]:
Sim.add_strategy("ETH", 100, 2000, 2000, 0, 1000, 1000)["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,100.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,0.000000,0.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


We are trying to make the AMM buy ETH. That fails because it does not have any USD.

In [29]:
Sim.amm_buys("ETH", 10)

{'success': False,
 'error': 'token USDC has no non-empty liquidity positions',
 'exception': ValueError('token USDC has no non-empty liquidity positions')}

Now we are trying to make the AMM sell more ETH than it has. That also fails.

In [30]:
Sim.amm_sells("ETH", 101)

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

## First cycle, part A: sell ETH

Let's look at the order book first: we have 100 ETH for sale at 2000, and we buy ETH at 1000, but currently have no USDC to sell

In [31]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,100.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,0.000000,0.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


So as we've seen above, we can't buy ETH. Not even a tiny little bit.

In [32]:
Sim.amm_buys("ETH", 0.00001)

{'success': False,
 'error': 'token USDC has no non-empty liquidity positions',
 'exception': ValueError('token USDC has no non-empty liquidity positions')}

However, the AMM can sell say 10 ETH, at a price of 2000.

In [33]:
Sim.amm_sells("ETH", 10)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


Note that the 10 ETH that were sold against 20000 USDC now show up on the USDC curve where `y=20,000`. Also note that `y_int=20,000`, after we previously had `y_int=0`. This means the curve has been expanded to make space for the USDC received.

In [34]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,90.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,20000.000000,20000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


We've sold 10 above, so can we sell 90+epsilon? Obviously not

In [35]:
Sim.amm_sells("ETH", 90.0000001)

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

However, it can sell 90

In [36]:
Sim.amm_sells("ETH", 90)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #0,False,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,1,1,A,AMM sells 90ETH buys 180000USDC,True,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


Now the ETH curve is empty (`y=0`), but the USDC curve is loaded with `y=200,000` USDC (100*2000), so we went cycle down.

In [37]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,0.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,200000.000000,200000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


In [38]:
Sim.state()["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,A,AMM sells 10ETH buys 20000USDC,True,True,None,10.0,ETH,20000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH
0,1.0,1,0,route #0,False,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,1,1,A,AMM sells 90ETH buys 180000USDC,True,True,None,90.0,ETH,180000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


Can it now sell epsilon? No, obviously not. And neither can it sell zero

In [39]:
Sim.amm_sells("ETH", 0.0000001)

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

In [40]:
Sim.amm_sells("ETH", 0)

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

## First cycle, part B: sell USDC

Reminder, the current state of the system is 0 ETH and 200,000 USDC

In [41]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,100.000000,0.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,200000.000000,200000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


As we have just seen, there is no ETH left to sell

In [42]:
Sim.amm_sells("ETH", 0.0000001)

{'success': False,
 'error': 'token ETH has no non-empty liquidity positions',
 'exception': ValueError('token ETH has no non-empty liquidity positions')}

But of course we can sell USDC. Let's sell the whole bunch. Note that we sell it at a price of 1,000 USDC per ETH (because this is where the curve is)

In [43]:
Sim.amm_sells("USDC", 200000)["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #1,False,True,None,200000.0,USDC,200.0,ETH,ETHUSDC,1,1,1000.0,USDC per ETH
0,2,2,A,AMM sells 200000USDC buys 200ETH,True,True,None,200000.0,USDC,200.0,ETH,ETHUSDC,[1],1,1000.0,USDC per ETH


In [44]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,200.000000,200.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,200000.000000,0.000000,USDC,1000.000000,1000.000000,USDC per ETH,0
